In [2]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *

import numpy as np
import SimpleITK as sitk

import csv

In [3]:
#CREATE CSV FILE FOR NIFTI
#PATIENT NAME  |  STUDY UID  | CT PATH  |  PET PATH  |  MASK PATH 

csv_pet0_path = '/media/salim/DD 2To/AHL2011_CSV_DATA/pet0.csv'

with open(csv_pet0_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ';') #liste pour chaque ligne 
    csv_data = []
    for row in reader :
        csv_data.append(row)
        
del csv_data[0] #enlever première ligne

In [5]:
print(len(csv_data))
list_nifti = os.listdir('/media/salim/DD 2To/AHL2011_NIFTI')



796


In [32]:
all_liste = []
for i in range(len(csv_data)) : 
    subliste = []
    subliste.append(csv_data[i][0])
    study_uid = csv_data[i][2]
    subliste.append(study_uid)
    for nifti in list_nifti : 
        if study_uid in nifti : 
            subliste.append(nifti)

    all_liste.append(subliste)





In [43]:
new_liste = []
for liste in all_liste : 
    if len(liste) == 5 : 
        new_liste.append(liste)

In [44]:
print(len(new_liste))

518


In [ ]:
#enlever doublon 

In [18]:
list_nifti = os.listdir('/media/salim/DD 2To/AHL2011_NIFTI')
